In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


Fuzz module is used to compare the two given string. It returns a score out of 100 after comparison using the different methods.

In [2]:
!pip install fuzzywuzzy
!pip install python-Levenshtein

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 174 kB 30.5 MB/s 
     |████████████████████████████████| 2.2 MB 64.0 MB/s 


In [3]:
# Import libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [13]:
def data_processing(flipkart_data, amazon_data):
  # Columns required for comparing the products are: product name, retail price and discounted price
  # Extract only required columns
  flipkart_req_cols = pd.DataFrame(columns=["Product name in Flipkart", "Retail Price in Flipkart", "Discount Price in Flipkart"])
  flipkart_req_cols[["Product name in Flipkart", "Retail Price in Flipkart", "Discount Price in Flipkart"]] = flipkart_data[["product_name", "retail_price", "discounted_price"]]
  amazon_req_cols = pd.DataFrame(columns=["Product name in Amazon", "Retail Price in Amazon", "Discount Price in Amazon"])
  amazon_req_cols[["Product name in Amazon", "Retail Price in Amazon", "Discount Price in Amazon"]] = amazon_data[["product_name", "retail_price", "discounted_price"]]
  
  # Check for shape of dataset
  print("Shape of Flipkart data",flipkart_req_cols.shape)
  print("Shape of Amazon data",amazon_req_cols.shape)
  print("-------------------------------------------------------------------\n")
  
  # Check for null values
  print("Check for null values\n")
  print("-------------------------------------------------------------------\n")
  print("Flipkart data info", flipkart_req_cols.info())
  print("Amazon data info", amazon_req_cols.info())
  print("-------------------------------------------------------------------\n")
  print("Flipkart data contains null values hence drop those rows\n")
  print("-------------------------------------------------------------------\n")

  # Drop the rows containing null values
  print("Drop the rows with null values\n")
  print("-------------------------------------------------------------------\n")
  dropna_flipkart_data = flipkart_req_cols.dropna()
  dropna_amazon_data = amazon_req_cols.dropna()
  print("Flipkart data info after dropping null values", dropna_flipkart_data.info())
  print("Amazon data info after dropping null values", dropna_amazon_data.info())
  print("-------------------------------------------------------------------\n")

  # Delete all rows with column 'Retail Price' having negative values
  print("Retain data with positive 'Retail Price' as price cannot be negative\n")
  print("-------------------------------------------------------------------\n")
  drop_neg_flipkart = dropna_flipkart_data[(dropna_flipkart_data['Retail Price in Flipkart'] >= 0)].reset_index()
  drop_neg_amazon = dropna_amazon_data[(dropna_amazon_data['Retail Price in Amazon'] >= 0)].reset_index()
  print("Processed Flipkart data", drop_neg_flipkart)
  print("Processed Amazon data", drop_neg_amazon)
  print("-------------------------------------------------------------------\n")
  print("                          PROCESSED DATA                          \n")
  print("-------------------------------------------------------------------\n")

  processed_flipkart_data = drop_neg_flipkart.drop(["index"],axis=1)
  processed_amazon_data = drop_neg_amazon.drop(["index"],axis=1)

  return processed_flipkart_data, processed_amazon_data

# Load flipkart and amazon dataset

flipkart_data = pd.read_csv("/gdrive/My Drive/shack labs/DS - Assignment Part 2 data set/flipkart_com-ecommerce_sample.csv", encoding = 'unicode_escape') # Load Flipkart data
amazon_data = pd.read_csv('/gdrive/My Drive/shack labs/DS - Assignment Part 2 data set/amz_com-ecommerce_sample.csv', encoding = 'unicode_escape') # Load Amazon data
# Data Pre-processing
processed_data = data_processing(flipkart_data, amazon_data)
print(processed_data)

Shape of Flipkart data (20000, 3)
Shape of Amazon data (20000, 3)
-------------------------------------------------------------------

Check for null values

-------------------------------------------------------------------

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 3 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Product name in Flipkart    20000 non-null  object 
 1   Retail Price in Flipkart    19922 non-null  float64
 2   Discount Price in Flipkart  19922 non-null  float64
dtypes: float64(2), object(1)
memory usage: 468.9+ KB
Flipkart data info None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 3 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Product name in Amazon    20000 non-null  object
 1   Retail Price in Amazon    2

In [16]:
import string
import re

def preprocessing_text(text_data):
  text = text_data.lower()
  english_punctuations = string.punctuation
  punctuations_list = english_punctuations
  translator = str.maketrans('', '', punctuations_list)
  # Remove stop words mentioned in the below list
  stopwordlist = ['a', 'about', 'above', 'after', 'again', 'ain', 'all', 'am', 'an',
             'and','any','are', 'as', 'at', 'be', 'because', 'been', 'before',
             'being', 'below', 'between','both', 'by', 'can', 'd', 'did', 'do',
             'does', 'doing', 'down', 'during', 'each','few', 'for', 'from',
             'further', 'had', 'has', 'have', 'having', 'he', 'her', 'here',
             'hers', 'herself', 'him', 'himself', 'his', 'how', 'i', 'if', 'in',
             'into','is', 'it', 'its', 'itself', 'just', 'll', 'm', 'ma',
             'me', 'more', 'most','my', 'myself', 'now', 'o', 'of', 'on', 'once',
             'only', 'or', 'other', 'our', 'ours','ourselves', 'out', 'own', 're',
             'rt', 's', 'same', 'she', "shes", 'should', "shouldve",'so', 'some', 'such',
             't', 'than', 'that', "thatll", 'the', 'their', 'theirs', 'them',
             'themselves', 'then', 'there', 'these', 'they', 'this', 'those',
             'through', 'to', 'too','under', 'until', 'up', 've', 'very', 'was',
             'we', 'were', 'what', 'when', 'where','which','while', 'who', 'whom',
             'why', 'will', 'with', 'won', 'y', 'you', "youd","youll", "youre",
             "youve", 'your', 'yours', 'yourself', 'yourselves']
  STOPWORDS = set(stopwordlist)
  text = " ".join([word for word in str(text).split() if word not in STOPWORDS])
  # Remove words attached with numbers
  text = re.sub(r"[A-Za-z]+\d+|\d+[A-Za-z]+",'', text)
  # Remove puntuations
  text = text.translate(translator)
  # Remove Numbers
  text = re.sub('[0-9]+', '', text)
  # Remove unwanted characters
  text = re.sub(r'[^\w]', ' ', text)
  return text

In [17]:
# Match the product name in flipkart and amazon for price comparison
# Matching strings is carried out using fuzzywuzzy package  

def product_match(processed_data, product_name):
  prod_index_f = []
  prod_index_a = []
  for i in range(2):
    for j in processed_data[i]:
      if j == "Product name in Flipkart":
        # print(j)
        count_f = 0
        for k in processed_data[i][j]:

          english_punctuations = string.punctuation
          punctuations_list = english_punctuations
          translator = str.maketrans('', '', punctuations_list)

          processed_text = preprocessing_text(k)
          processed_prod_name = preprocessing_text(product_name)
          fuzz_ratio = fuzz.ratio(processed_text,processed_prod_name)
          # print(fuzz_ratio)
          count_f += 1
          if fuzz_ratio > 85:        
            prod_index_f.append(count_f - 1)
            # print(prod_index)
            matching_prod_flipkart = processed_data[i].loc[prod_index_f]
            # print(matching_prod_flipkart)
            break
      elif j == "Product name in Amazon":
        count_a = 0
        for k in processed_data[i][j]:
          # print(j)
          english_punctuations = string.punctuation
          punctuations_list = english_punctuations
          translator = str.maketrans('', '', punctuations_list)
          processed_text = preprocessing_text(k)
          processed_prod_name = preprocessing_text(product_name)

          fuzz_ratio = fuzz.ratio(processed_text,processed_prod_name)
          # print(fuzz_ratio)
          count_a += 1
          if fuzz_ratio > 85:        
            prod_index_a.append(count_a - 1)
            # print(prod_index)
            matching_prod_amazon = processed_data[i].iloc[prod_index_a]
            # print(matching_prod_amazon)
            break
  merged_data = pd.concat([matching_prod_flipkart, matching_prod_amazon], axis=1)
  return merged_data

In [24]:
prod_name = input("Enter Product Name:\n")
product_matching = product_match(processed_data, prod_name)
product_matching

Enter Product Name:
icons All Purpose Arnica Dog Shampoo


,Product name in Flipkart,Retail Price in Flipkart,Discount Price in Flipkart,Product name in Amazon,Retail Price in Amazon,Discount Price in Amazon
4,Sicons All Purpose Arnica Dog Shampoo,220.0,210.0,Sicons All Purpose Arnica Dog Shampoo,208,258


fuzz.ratio() is one of the important methods of fuzz module. It compares the string and score on the basis of how much the given string are matched.